# 1. Setup ViZDoom

In [1]:
!pip install vizdoom

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!cd github_clone & git clone https://github.com/Farama-Foundation/ViZDoom

fatal: destination path 'ViZDoom' already exists and is not an empty directory.


In [3]:
# ViZDoom for game env
from vizdoom import *
# Random for action sampling
import random
# Time for sleeping
import time
# Identity matrix
import numpy as np

In [4]:
# Setup game
game = DoomGame()
game.load_config('github_clone/ViZDoom/scenarios/basic.cfg')
game.init()

In [5]:
# Define the set of actions within this Doom env
actions = np.identity(3, dtype=np.uint8)

In [6]:
episodes = 10

for episode in range(episodes):
    # Each episode is a replay of a game
    game.new_episode()
    
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        # Ammo
        info = state.game_variables
        # Take action, skip next 4 frames, and return the result (better for our agent to see the action take place)
        reward = game.make_action(random.choice(actions), 4)
        print('reward: ', reward)
        time.sleep(0.02)
    print('Result: ', game.get_total_reward())
    time.sleep(2)

reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -9.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -9.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -9.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -9.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0
reward:  -4.0


ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.

In [ ]:
game.close()

# 2. Convert to Gym Environment

In [ ]:
!pip install gym

In [7]:
# Env base class
from gym import Env
# Gym spaces
# Discrete: Represents our actions
# Box: An array of any shape
from gym.spaces import Discrete, Box
# OpenCV
import cv2

In [8]:
# ViZDoom env
class VizDoomGym(Env):
    
    # Start the env
    def __init__(self, render=False):
        # Inherit from our Env class
        super().__init__()
        # Setup game
        self.game = DoomGame()
        self.game.load_config('github_clone/ViZDoom/scenarios/basic.cfg')
        
        # Allow us to train the model without having to open a new window each time
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        self.game.init()
        
        # Create spaces
        # game.get_state().screen_buffer.shape
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8)
        self.action_space = Discrete(3)
        
    # Represents a step in the env
    def step(self, action):
        # Specify action and take step
        actions = np.identity(3)
        reward = self.game.make_action(actions[action], 4)
        
        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0
            
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info
    
    # Render the env
    def render():
        pass
    
    # New game
    def reset(self):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it
    # Reshaping the array into the form that cvtColor expects it in (channel should be at the end)
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation,0,-1), cv2.COLOR_BGR2GRAY)
        # Scales it down so less pixels to process
        # Reducing what does not bring value when come time to train the model
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        
        return state
    
    # Close the env
    def close(self):
        self.game.close()

In [ ]:
env = VizDoomGym(render=True)

In [ ]:
env.close()

In [9]:
# Env checker
from stable_baselines3.common import env_checker

In [ ]:
env_checker.check_env(env)

In [ ]:
!pip install matplotlib
from matplotlib import pyplot as plt
#plt.imshow(state)

# 3. Setup Callback

In [10]:
!pip install torch torchvision torchaudio
# PPO for RL
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
# OS for file
import os
# Callback
from stable_baselines3.common.callbacks import BaseCallback

In [12]:
# Save our model every x number of steps
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [13]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [14]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

# 4. Train the Model

In [15]:
# PPO
from stable_baselines3 import PPO

In [16]:
# No render
env = VizDoomGym()

In [19]:
# Check 'scenarios/basic.cfg' in the ViZDoom GitHub repository and put steps somewhat to the maximum actions defined in a single game
# The agent will take 2048 sets of observations, actions, log probs, values store in the buffer for one iteration 
# Such a high steps to decrease the clip range value
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.0001, n_steps=2048)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./logs/log_basic\PPO_11
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 26.9     |
|    ep_rew_mean     | -47.9    |
| time/              |          |
|    fps             | 14       |
|    iterations      | 1        |
|    time_elapsed    | 141      |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 28           |
|    ep_rew_mean          | -47.1        |
| time/                   |              |
|    fps                  | 12           |
|    iterations           | 2            |
|    time_elapsed         | 331          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0049674455 |
|    clip_fraction        | 0.147        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.87        |
|    ep_rew_mean          | 59          |
| time/                   |             |
|    fps                  | 10          |
|    iterations           | 11          |
|    time_elapsed         | 2143        |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.020391311 |
|    clip_fraction        | 0.434       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.806      |
|    explained_variance   | 0.44        |
|    learning_rate        | 0.0001      |
|    loss                 | 610         |
|    n_updates            | 100         |
|    policy_gradient_loss | 0.031       |
|    value_loss           | 1.15e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.92  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4.1        |
|    ep_rew_mean          | 86.4       |
| time/                   |            |
|    fps                  | 9          |
|    iterations           | 21         |
|    time_elapsed         | 4554       |
|    total_timesteps      | 43008      |
| train/                  |            |
|    approx_kl            | 0.04241727 |
|    clip_fraction        | 0.136      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.275     |
|    explained_variance   | 0.72       |
|    learning_rate        | 0.0001     |
|    loss                 | 38.9       |
|    n_updates            | 200        |
|    policy_gradient_loss | 0.0214     |
|    value_loss           | 110        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4.51        |
|    ep_rew_m

# 5. Test the Model

In [ ]:
# Evaluation policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
#model = PPO.load('./train/train_basic/best_model_100000')

In [ ]:
#env = VizDoomGym(render=True)

In [ ]:
#mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100)

In [ ]:
#mean_reward

In [ ]:
#model.predict(obs)

In [ ]:
#for episode in range(5):
    #observation = env.reset()
    #done = False
    #total_reward = 0
    
    #while not done:
        #action, _ = model.predict(observation)
        #observation, reward, done, info = env.step(action)
        # time.sleep(0.20)
        #total_reward += reward
    #print('Total Reward for episode {} is {}'.format(episode, total_reward))
    #time.sleep(2)